# Draft Inundation duration notebook

**What does this notebook do?** 

**Requirements:**

You need [data access](http://geoscienceaustralia.github.io/digitalearthau/connect/account.html#data-access) to the following [NCI projects](http://geoscienceaustralia.github.io/digitalearthau/connect/account.html#nci-account-registration) to run this notebook: `rs0` 

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

**Date:** July 2019

**Author:** Bex Dunn

#Fixme: update tags index

**Tags**: :index:`ITEM`, :index:`ITEM_confidence`, :index:`intertidal_zone`, :index:`masking`, :index:`query`, :index:`dc.load`, :index:`plot`

### load modules

In [10]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
from shapely import geometry
import seaborn as sns
import sys
import xarray as xr

from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from textwrap import wrap

from datacube.storage import masking
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from digitalearthau.utils import wofs_fuser

sys.path.append('/g/data/r78/rjd547/jupyter_notebooks/dea-notebooks/10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, WetlandsTools

dc = datacube.Datacube(app='wetlands insight tool')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## define custom functions only used in this notebook

### specify input and output paths

In [11]:
#path to hydrograph data
flowdata = '/g/data/r78/rjd547/CEWO_Project/Data/flow_data_mac_marsh_2018/FlowData.xlsx'

In [12]:
#set output directory
global Output_dir
Output_dir = '/g/data/r78/rjd547/CEWO_Project/prototyping_outputs/'

In [13]:
#set polygon path
poly_path='/g/data/r78/rjd547/Ramsar_Wetlands/shapefiles/ExplodedRAMSAR.shp'

### define an area to work in

In [14]:
# # Set up analysis data query using a buffer around a lat-long point
# lat, lon, buffer = -30.72360833, 147.53250000, 5000
# x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
# query = {'x': (x - buffer, x + buffer),
#          'y': (y - buffer, y + buffer),    
#          #'time': ('2015-01-01', '2018-07-30'),
#          'time': ('1987-01-01', '2019-01-01'),
#          'crs': 'EPSG:3577'}


## get feature from shapefile to construct query

In [15]:
with fiona.open(poly_path) as shapes:
    #get crs object from polygon file
    crs = geometry.CRS(shapes.crs_wkt)
    for feature in shapes:
        #print (feature['id'])
        if feature['id'] == str(94): #matching this because I know the number for mac marshes
            #get feature geometry
            feature_geometry=feature['geometry']
            polyname_test =WetlandsTools.get_RAMSAR_polyName(feature)
            print(polyname_test)
            #create datacube geometry.Geometry object
            geom = geometry.Geometry(feature_geometry, crs=crs)
            query = {'geopolygon': geom}#, 'time': ('2001-01-01', '2003-01-01') }# this should run for all time, if there is no time set?
            #load and mask data. selecting data with more than 90% clear for the geobox around the polygon... #FIXME

processing polygon The_Macquarie_Marshes-Macquarie_Marshes_Nature_Reserve-NSW-94
The_Macquarie_Marshes-Macquarie_Marshes_Nature_Reserve-NSW-94


### show our area of interest on a map

In [16]:
# get the bounding box around the polygon in order to plot our area of interest
feature_bounding_box = geometry.Geometry(feature_geometry, crs=crs).boundingbox

In [17]:
map = DEAPlotting.display_map(y=(feature_bounding_box[1], feature_bounding_box[3]), x = (feature_bounding_box[0], feature_bounding_box[2]))

In [18]:
map

In [ ]:
import folium
from pyproj import Proj, transform
import math

# Define function to assist `display_map` in selecting a zoom level for plotting
def _degree_to_zoom_level(l1, l2, margin = 0.0):
    
    """
    Helper function to set zoom level for `display_map`
    """
    
    degree = abs(l1 - l2) * (1 + margin)
    zoom_level_int = 0
    if degree != 0:
        zoom_level_float = math.log(360 / degree) / math.log(2)
        zoom_level_int = int(zoom_level_float)
    else:
        zoom_level_int = 18
    return zoom_level_int

# def display_map(y, x, crs='EPSG:3577', margin=-0.5, zoom_bias=0):
y=(feature_bounding_box[1], feature_bounding_box[3])
x = (feature_bounding_box[0], feature_bounding_box[2])
crs='EPSG:3577'
margin=-0.5
zoom_bias=0

""" 
Given a set of x and y coordinates, this function generates an interactive map with a bounded 
rectangle overlayed on Google Maps imagery.        

Last modified: November 2018
Author: Robbi Bishop-Taylor

Modified from function written by Otto Wagner available here: 
https://github.com/ceos-seo/data_cube_utilities/tree/master/data_cube_utilities

Parameters
----------  
x : (float, float)
    A tuple of x coordinates in (min, max) format. 
y : (float, float)
    A tuple of y coordinates in (min, max) format.
crs : string, optional
    A string giving the EPSG CRS code of the supplied coordinates. The default is 'EPSG:3577'.
margin : float
    A numeric value giving the number of degrees lat-long to pad the edges of the rectangular overlay 
    polygon. A larger value results more space between the edge of the plot and the sides of the polygon.
    Defaults to -0.5.
zoom_bias : float or int
    A numeric value allowing you to increase or decrease the zoom level by one step. Defaults to 0; set
    to greater than 0 to zoom in, and less than 0 to zoom out.

Returns
-------
folium.Map : A map centered on the supplied coordinate bounds. A rectangle is drawn on this map detailing 
the perimeter of the x, y bounds.  A zoom level is calculated such that the resulting viewport is the
closest it can possibly get to the centered bounding rectangle without clipping it. 
"""
    
# Convert each corner coordinates to lat-lon
all_x = (x[0], x[1], x[0], x[1])
all_y = (y[0], y[0], y[1], y[1])        
all_longitude, all_latitude = transform(Proj(init=crs), Proj(init='EPSG:4326'), all_x, all_y) 

# Calculate zoom level based on coordinates 
lat_zoom_level = _degree_to_zoom_level(min(all_latitude), max(all_latitude), margin = margin) + zoom_bias
lon_zoom_level = _degree_to_zoom_level(min(all_longitude), max(all_longitude), margin = margin) + zoom_bias
zoom_level = min(lat_zoom_level, lon_zoom_level) 

# Identify centre point for plotting
center = [np.mean(all_latitude), np.mean(all_longitude)]

# Create map
interactive_map = folium.Map(location=center,
                             zoom_start=zoom_level,
                             tiles="http://mt1.google.com/vt/lyrs=y&z={z}&x={x}&y={y}",
                             attr="Google") 

# Create bounding box coordinates to overlay on map
line_segments = [(all_latitude[0], all_longitude[0]),
                 (all_latitude[1], all_longitude[1]),
                 (all_latitude[3], all_longitude[3]),
                 (all_latitude[2], all_longitude[2]),
                 (all_latitude[0], all_longitude[0])] 

# Add bounding box as an overlay
interactive_map.add_child(folium.features.PolyLine(locations=line_segments,
                                                   color='red', opacity=0.8))

       
map_poly_geom =geom.to_crs(CRS('epsg:4326'))
# geom = folium.GeoJson(gg.__geo_interface__) 
interactive_map.add_child(folium.GeoJson(map_poly_geom.__geo_interface__))

In [ ]:
map_poly_geom =geom.to_crs(CRS('epsg:4326'))

In [19]:
def display_map(y, x, crs='EPSG:3577', margin=-0.5, zoom_bias=0):
    
    """ 
    Given a set of x and y coordinates, this function generates an interactive map with a bounded 
    rectangle overlayed on Google Maps imagery.        
    
    Last modified: November 2018
    Author: Robbi Bishop-Taylor
    
    Modified from function written by Otto Wagner available here: 
    https://github.com/ceos-seo/data_cube_utilities/tree/master/data_cube_utilities
    
    Parameters
    ----------  
    x : (float, float)
        A tuple of x coordinates in (min, max) format. 
    y : (float, float)
        A tuple of y coordinates in (min, max) format.
    crs : string, optional
        A string giving the EPSG CRS code of the supplied coordinates. The default is 'EPSG:3577'.
    margin : float
        A numeric value giving the number of degrees lat-long to pad the edges of the rectangular overlay 
        polygon. A larger value results more space between the edge of the plot and the sides of the polygon.
        Defaults to -0.5.
    zoom_bias : float or int
        A numeric value allowing you to increase or decrease the zoom level by one step. Defaults to 0; set
        to greater than 0 to zoom in, and less than 0 to zoom out.
        
    Returns
    -------
    folium.Map : A map centered on the supplied coordinate bounds. A rectangle is drawn on this map detailing 
    the perimeter of the x, y bounds.  A zoom level is calculated such that the resulting viewport is the
    closest it can possibly get to the centered bounding rectangle without clipping it. 
    """
    
    # Convert each corner coordinates to lat-lon
    all_x = (x[0], x[1], x[0], x[1])
    all_y = (y[0], y[0], y[1], y[1])        
    all_longitude, all_latitude = transform(Proj(init=crs), Proj(init='EPSG:4326'), all_x, all_y) 

    # Calculate zoom level based on coordinates 
    lat_zoom_level = _degree_to_zoom_level(min(all_latitude), max(all_latitude), margin = margin) + zoom_bias
    lon_zoom_level = _degree_to_zoom_level(min(all_longitude), max(all_longitude), margin = margin) + zoom_bias
    zoom_level = min(lat_zoom_level, lon_zoom_level) 

    # Identify centre point for plotting
    center = [np.mean(all_latitude), np.mean(all_longitude)]

    # Create map
    interactive_map = folium.Map(location=center,
                                 zoom_start=zoom_level,
                                 tiles="http://mt1.google.com/vt/lyrs=y&z={z}&x={x}&y={y}",
                                 attr="Google") 

#     # Create bounding box coordinates to overlay on map
#     line_segments = [(all_latitude[0], all_longitude[0]),
#                      (all_latitude[1], all_longitude[1]),
#                      (all_latitude[3], all_longitude[3]),
#                      (all_latitude[2], all_longitude[2]),
#                      (all_latitude[0], all_longitude[0])] 
    
#     # Add bounding box as an overlay
#     interactive_map.add_child(folium.features.PolyLine(locations=line_segments,
#                                                        color='red', opacity=0.8))
    
    interactive_map.add_child(folium.GeoJson(map_poly_geom.__geo_interface__))

    # Add clickable lat-lon popup box
    interactive_map.add_child(folium.features.LatLngPopup())        

    return interactive_map

In [22]:
args= {'overlay':None}

In [23]:
DEAPlotting.display_map(y=(feature_bounding_box[1], feature_bounding_box[3]), x = (feature_bounding_box[0], feature_bounding_box[2]), **args)

TypeError: display_map() got an unexpected keyword argument 'overlay'